<a href="https://colab.research.google.com/github/ASIF-Mahmud1/MachineLearningProjects/blob/bert/BERT/ClassificationWithBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
## install tensorflow-text
%%capture
%%bash

pip install -U tensorflow-text

In [48]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

## Import Diminsihing Data

In [49]:
url="https://raw.githubusercontent.com/ASIF-Mahmud1/Exploration/text-classifier/DiminishingTerms/dataSet-2.csv"
from io import StringIO
import string
import pandas as pd
import requests
s=requests.get(url).text

message_data=pd.read_csv(StringIO(s))
message_data['tag'] = message_data['tag'].str.strip()

message_data.head()
message_data['tag'].describe()

count             514
unique              2
top       diminishing
freq              336
Name: tag, dtype: object

In [50]:
message_data.groupby('tag').describe()


id                                                           
             count        mean         std  min     25%    50%     75%    max
tag                                                                          
diminishing  336.0  220.550595  118.560988  1.0  121.75  205.5  332.25  416.0
strong       178.0  324.477528  173.697939  2.0  224.50  424.5  468.75  513.0

In [51]:
message_data['value']=message_data['tag'].apply(lambda x: 1 if x=='strong' else 0)
message_data.head()

,id,tag,sentence,value
0,1,diminishing,I�m no expert but �,0
1,2,strong,This is what I see,1
2,3,diminishing,It�s just my opinion �,0
3,4,strong,It�s my opinion�,1
4,5,diminishing,Just checking in �,0


##Split it into training and test data set

In [52]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(message_data['sentence'],message_data['value'],test_size=1)

In [53]:
X_train.head(4)
len(X_train)

513

### Now lets import BERT model and get embeding vectors for few sample statements

In [45]:
# get encoder-url and corresponding preprocess url from: https://tfhub.dev/google/collections/bert/1

preprocess_url = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
encoder_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4"

bert_preprocess = hub.KerasLayer(preprocess_url)
bert_encoder = hub.KerasLayer(encoder_url)

In [46]:
## Observation 

def get_sentence_embeding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

get_sentence_embeding([
    "500$ discount. hurry up", 
    "Bhavin, are you up for a volleybal game tomorrow?"]
)

<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-0.84351707, -0.5132727 , -0.88845736, ..., -0.74748856,
        -0.7531474 ,  0.91964495],
       [-0.8720837 , -0.5054399 , -0.94446677, ..., -0.8584751 ,
        -0.71745366,  0.88082993]], dtype=float32)>

###Get embeding vectors for few sample words. Compare them using cosine similarity

In [54]:
#Observation

e = get_sentence_embeding([
    "banana", 
    "grapes",
    "mango",
    "jeff bezos",
    "elon musk",
    "bill gates"
]
)

In [55]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity([e[0]],[e[1]])            #Observation

#Values near to 1 means they are similar. 0 means they are very different. Above you can use comparing "banana" vs "grapes" you get 0.99 similarity as they both are fruits

array([[0.9911089]], dtype=float32)

In [56]:
cosine_similarity([e[0]],[e[3]])
# Comparing banana with jeff bezos you still get 0.84 but it is not as close as 0.99 that we got with grapes

cosine_similarity([e[3]],[e[4]])
# Jeff bezos and Elon musk are more similar then Jeff bezos and banana as indicated above

array([[0.98720354]], dtype=float32)

Build Model
There are two types of models you can build in tensorflow.

(1) Sequential (2) Functional

So far we have built sequential model. But below we will build functional model. More information on these two is here: https://becominghuman.ai/sequential-vs-functional-model-in-keras-20684f766057

In [57]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [58]:
model.summary()


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer_2 (KerasLayer)      {'input_mask': (None 0           text[0][0]                       
__________________________________________________________________________________________________
keras_layer_3 (KerasLayer)      {'encoder_outputs':  109482241   keras_layer_2[0][0]              
                                                                 keras_layer_2[0][1]              
                                                                 keras_layer_2[0][2]              
____________________________________________________________________________________________

In [59]:
len(X_train)


513

In [60]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

### Train Model

In [61]:
model.fit(X_train, y_train, epochs=5)


Epoch 1/5
17/17 [==============================] - 230s 13s/step - loss: 0.6712 - accuracy: 0.6160
Epoch 2/5
17/17 [==============================] - 221s 13s/step - loss: 0.6419 - accuracy: 0.6608
Epoch 3/5
17/17 [==============================] - 221s 13s/step - loss: 0.6153 - accuracy: 0.6413
Epoch 4/5
17/17 [==============================] - 221s 13s/step - loss: 0.6064 - accuracy: 0.6569
Epoch 5/5
17/17 [==============================] - 221s 13s/step - loss: 0.6087 - accuracy: 0.6530


In [62]:
model.evaluate(X_test, y_test)


1/1 [==============================] - 2s 2s/step - loss: 0.3876 - accuracy: 1.0000


[0.3876095414161682, 1.0]

Inference

In [66]:

reviews = [
           "This is unclear",
   "Our success is inevitable",
"I will definitely pass the exam",
"I am giving up",
"I don't know, if I will pass the exam",
"Maybe I will pass the exam",

]
model.predict(reviews)

array([[0.4406702 ],
       [0.402784  ],
       [0.36375138],
       [0.37997845],
       [0.17167109],
       [0.37352258]], dtype=float32)